In [ ]:
%pip install numpy pandas tensorflow-cpu

In [1]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers

import os.path

2024-03-21 19:42:28.272678: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 19:42:28.274522: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 19:42:28.299981: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 19:42:28.913202: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/workspaces/free-code-camp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get data files
if not os.path.isfile('train-data.tsv'):
	!wget 'https://cdn.freecodecamp.org/project-data/sms/train-data.tsv'
if not os.path.isfile('valid-data.tsv'):
	!wget 'https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv'

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
# load data into dataframes
df_train = pd.read_csv(train_file_path, sep='\t', names=['type', 'message'])
df_test = pd.read_csv(test_file_path, sep='\t', names=['type', 'message'])

display(df_train.tail())
display(df_test.tail())

,type,message
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...
4178,ham,tell my bad character which u dnt lik in me. ...


,type,message
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...
1391,spam,not heard from u4 a while. call 4 rude chat pr...


In [4]:
# encode type column
type_encoder = layers.StringLookup(vocabulary=df_train['type'].unique(), num_oov_indices=0)

df_train['type'] = pd.Series(type_encoder(df_train['type']), dtype='int32')
df_test['type'] = pd.Series(type_encoder(df_test['type']), dtype='int32')

display(type_encoder.get_vocabulary())
display(df_train.tail())
display(df_test.tail())

['ham', 'spam']

,type,message
4174,0,just woke up. yeesh its late. but i didn't fal...
4175,0,what do u reckon as need 2 arrange transport i...
4176,1,free entry into our £250 weekly competition ju...
4177,1,-pls stop bootydelious (32/f) is inviting you ...
4178,0,tell my bad character which u dnt lik in me. ...


,type,message
1387,0,true dear..i sat to pray evening and felt so.s...
1388,0,"what will we do in the shower, baby?"
1389,0,where are you ? what are you doing ? are yuou ...
1390,1,ur cash-balance is currently 500 pounds - to m...
1391,1,not heard from u4 a while. call 4 rude chat pr...


In [5]:
# create vocabulary
messages_combined = pd.Series(df_train['message'].str.cat(sep=' ').split())
unique_word_count = pd.Series(messages_combined.value_counts())

# get max length
longest_message_length = int(df_train['message'].map(lambda message: len(message)).max())

def standardize_text(input):
	lowercase = tf.strings.lower(input)
	punctuation = tf.strings.regex_replace(input=input, pattern=r'\W', rewrite=' ')
	return punctuation


# encode messages
text_vectorizer = layers.TextVectorization(
	max_tokens=10000,
	output_mode='int',
	output_sequence_length=longest_message_length,
	standardize=standardize_text
)

# make text vectorizer to encode data
text_vectorizer.adapt(df_train['message'])

display(df_train['message'][0])
display(' '.join([text_vectorizer.get_vocabulary()[word] for word in text_vectorizer(df_train['message'][0])]))

'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'

'ahhhh just woken up had a bad dream about u tho so i dont like u right now i didnt know anything about comedy night but i guess im up for it                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [6]:
model = tf.keras.Sequential([
    text_vectorizer,
    layers.Embedding(
        input_dim=len(text_vectorizer.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    layers.Bidirectional(tf.keras.layers.LSTM(64)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# compile model
model.compile(
	loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
	optimizer=tf.keras.optimizers.Adam(1e-4),
	metrics=['accuracy']
)

In [7]:
# train model
model.fit(
	x=df_train['message'],
	y=df_train['type'],
	epochs=5,
)

Epoch 1/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 26s 179ms/step - accuracy: 0.8688 - loss: 0.6713
Epoch 2/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 179ms/step - accuracy: 0.8635 - loss: 0.4034
Epoch 3/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 181ms/step - accuracy: 0.9261 - loss: 0.1582
Epoch 4/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 182ms/step - accuracy: 0.9913 - loss: 0.0620
Epoch 5/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 181ms/step - accuracy: 0.9915 - loss: 0.0445


In [8]:
model.evaluate(df_test['message'], df_test['type'])

44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.9908 - loss: 0.0449


[0.05022852122783661, 0.9870689511299133]

In [9]:
data = np.array([
	"how are you doing today",
	"sale today! to stop texts call 98912460324",
], dtype='object')

predictions = model(data)
display(predictions)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-4.865856 ],
       [ 1.4405215]], dtype=float32)>

In [10]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
	prediction = model.predict(np.array([pred_text], dtype=df_test['message'].dtype))
	result = [prediction[0][0], 'ham' if prediction[0][0] <= 0.5 else 'spam']
	print(result)

	return result

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
# print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
[-4.8658566, 'ham']


In [11]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[-4.8658566, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[1.4405214, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[-10.314612, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[1.3010736, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
[3.4338818, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[-10.203658, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
[-9.4663725, 'ham']
You passed the challenge. Great job!
